In [29]:
import pandas as pd 
import matplotlib.pyplot as plt 

import numpy as np 

## Using Dianping dataset 

In [30]:
# users_df = pd.DataFrame(users)
# users_df.to_csv('data/dianping/users.csv', index=False)

# Convert rating.txt to interactions.csv
ratings = []
with open('../../Dianping/rating.txt', 'r') as f:
    for line in f:
        user_id, item_id, rating, date = line.strip().split('|')
        ratings.append({'user_id': user_id, 'item_id': item_id, 'rating': rating, 'timestamp': date})

ratings_df = pd.DataFrame(ratings)

In [31]:
ratings_df

,user_id,item_id,rating,timestamp
0,59708,0,3,2012-06-16
1,3781,1,4,2009-12-24
2,120358,2,4,2010-08-06
3,55553,3,5,2013-07-28
4,20837,4,4,2012-03-10
...,...,...,...,...
2149670,123340,2377,4,2009-11-15
2149671,61472,7027,3,2012-06-13
2149672,80236,1298,3,2011-03-08
2149673,936,6896,3,2012-09-16


In [32]:
user_txt_path = '../../Dianping/user.txt'
rating_txt_path = '../../Dianping/rating.txt'

rating_df = pd.read_csv(rating_txt_path, sep = '|', names = ['user_id', 'item_id', 'rating', 'date'])
user_df = pd.read_csv(user_txt_path, header=None, sep='|', names = ['user_id', 'friends_id'])
user_df['friends_id'] = user_df['friends_id'].apply(lambda x: list(map(int, x.split())) if pd.notna(x) else []) 
user_df.head(2), rating_df.head(2)

(   user_id                                         friends_id
 0        0  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
 1       29                                               [30],
    user_id  item_id  rating        date
 0    59708        0       3  2012-06-16
 1     3781        1       4  2009-12-24)

## Need to convert to atomic files for recbole compatibility

In [33]:
user_df.fillna(0)
rating_df.fillna(0)

,user_id,item_id,rating,date
0,59708,0,3,2012-06-16
1,3781,1,4,2009-12-24
2,120358,2,4,2010-08-06
3,55553,3,5,2013-07-28
4,20837,4,4,2012-03-10
...,...,...,...,...
2149670,123340,2377,4,2009-11-15
2149671,61472,7027,3,2012-06-13
2149672,80236,1298,3,2011-03-08
2149673,936,6896,3,2012-09-16


In [34]:
rating_df['date'] = pd.to_datetime(rating_df['date'])
rating_df['date']

0         2012-06-16
1         2009-12-24
2         2010-08-06
3         2013-07-28
4         2012-03-10
             ...    
2149670   2009-11-15
2149671   2012-06-13
2149672   2011-03-08
2149673   2012-09-16
2149674   2013-07-28
Name: date, Length: 2149675, dtype: datetime64[ns]

In [35]:
rating_df['date'] = rating_df['date'].values.astype(np.int64)//10**9
rating_df = rating_df.rename(columns={'date':'timestamp'})
rating_df.head() 

,user_id,item_id,rating,timestamp
0,59708,0,3,1339804800
1,3781,1,4,1261612800
2,120358,2,4,1281052800
3,55553,3,5,1374969600
4,20837,4,4,1331337600


In [36]:
rating_df['rating']

0          3
1          4
2          4
3          5
4          4
          ..
2149670    4
2149671    3
2149672    3
2149673    3
2149674    5
Name: rating, Length: 2149675, dtype: int64

In [37]:
# new_cols = []
# dfs = [user_df, rating_df]
# new_dfs = []
# for df in dfs:
#     for col in df.columns:
#         new_cols.append(f'{col}:{type(df[col][0]).__name__}')
#         df = df.rename(columns={col:new_cols[-1]})
#     new_dfs.append(df)

In [38]:
rating_df.head(2), user_df.head(2)

(   user_id  item_id  rating   timestamp
 0    59708        0       3  1339804800
 1     3781        1       4  1261612800,
    user_id                                         friends_id
 0        0  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
 1       29                                               [30])

In [39]:
rating_df = rating_df.rename(columns={'user_id':'user_id:token', 'item_id':'item_id:token', 'rating':'rating:float', 'timestamp':'timestamp:float'})
user_df = user_df.rename(columns={'user_id':'user_id:token', 'friends_id':'friends_id:token_seq'})

In [40]:
rating_df.head(2), user_df.head(2)

(   user_id:token  item_id:token  rating:float  timestamp:float
 0          59708              0             3       1339804800
 1           3781              1             4       1261612800,
    user_id:token                               friends_id:token_seq
 0              0  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
 1             29                                               [30])

In [41]:
# new_cols = []
# for col in user_df.columns:
#     new_cols.append(f'{col}:{type(user_df[col][0]).__name__}')
#     user_df = user_df.rename(columns={col:new_cols[-1]})

# new_cols = []

# for col in rating_df.columns:
#     new_cols.append(f'{col}:{type(rating_df[col][0]).__name__}')
#     rating_df = rating_df.rename(columns={col:new_cols[-1]})

In [42]:
# saving the datasets in atomic files format 
# user_df.to_csv('Dianping_local\Dianping_local.user', index=False)
rating_df.to_csv('Dianping_local\Dianping_local.inter', index=False, sep='\t')

In [10]:
# saving the datasets in .csv format 
user_df.to_csv('data/users.csv', index=False)
rating_df.to_csv('data/interactions.csv', index=False)

In [15]:
user_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147918 entries, 0 to 147917
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   user_id     147918 non-null  int64 
 1   friends_id  147918 non-null  object
dtypes: int64(1), object(1)
memory usage: 2.3+ MB


In [16]:
rating_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2149675 entries, 0 to 2149674
Data columns (total 4 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  int64 
 1   item_id  int64 
 2   rating   int64 
 3   date     object
dtypes: int64(3), object(1)
memory usage: 65.6+ MB
